<a href="https://colab.research.google.com/github/nkilaru213/ai-excercises/blob/main/NaveenKilaru_LLM_Data_A7_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **A7 - LLM Excercise**

# **Import Packages**

In [ ]:
# import the packages needed for colab
from google.colab import files

In [ ]:
# Step 1. Install Required Libraries
!pip install openai pandas

In [ ]:
# Step 2. Import Libraries
import openai
import os
import pandas as pd

from openai import OpenAI


In [ ]:
# Connect the the drive to access the google drive file system
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive



### ***Step 1: Preprocessing & Merging conditions and medications***

In [ ]:
# Load tables

# Read CONDITIONS data table from google drive file system
conditions = pd.read_csv("/content/drive/My Drive/Exercises/syntheticdata/conditions.csv")

# Read MEDICATIONS data table from google drive file system
medications = pd.read_csv("/content/drive/My Drive/Exercises/syntheticdata/medications.csv")

# Convert relevant date columns to datetime
conditions['START'] = pd.to_datetime(conditions['START'], errors='coerce')
medications['START'] = pd.to_datetime(medications['START'], errors='coerce')


# Display CONDITIONS table info
conditions.info()

# Display MEDICATIONS table info
medications.info()

# Join on PATIENT and ENCOUNTER to create a patient-visit level view
merged_df = pd.merge(conditions, medications, on=['PATIENT', 'ENCOUNTER'], how='inner', suffixes=('_COND', '_MED'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3876 entries, 0 to 3875
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   START        3876 non-null   datetime64[ns]
 1   STOP         2985 non-null   object        
 2   PATIENT      3876 non-null   object        
 3   ENCOUNTER    3876 non-null   object        
 4   SYSTEM       3876 non-null   object        
 5   CODE         3876 non-null   int64         
 6   DESCRIPTION  3876 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 212.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5605 entries, 0 to 5604
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   START              5605 non-null   datetime64[ns, UTC]
 1   STOP               5360 non-null   object             
 2   PATIENT            5605 non-nu

***Tables***

**medications**  : Stores patient medication details

**conditions** : Contains all details of the patient conditions

In [ ]:
# Display few rows from each dataframe
conditions.head()
medications.head()

,START,STOP,PATIENT,PAYER,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,PAYER_COVERAGE,DISPENSES,TOTALCOST,REASONCODE,REASONDESCRIPTION
0,2015-09-28 11:02:48+00:00,2015-10-15T09:04:48Z,30a6452c-4297-a1ac-977a-6a23237c7b46,d31fccc3-1767-390d-966a-22a5156f4219,953c5138-ce17-4084-3432-1ac23f184528,857005,Acetaminophen 325 MG / HYDROcodone Bitartrate ...,2.51,0.0,1,2.51,NaN,NaN
1,2015-09-28 11:02:48+00:00,2015-10-31T11:02:48Z,30a6452c-4297-a1ac-977a-6a23237c7b46,d31fccc3-1767-390d-966a-22a5156f4219,953c5138-ce17-4084-3432-1ac23f184528,310965,Ibuprofen 200 MG Oral Tablet,365.90,0.0,1,365.90,NaN,NaN
2,2005-11-08 20:24:07+00:00,NaN,34a4dcc4-35fb-6ad5-ab98-be285c586a4f,e03e23c9-4df1-3eb6-a62d-f70f02301496,d1cea2e5-1735-089f-c72f-22ad16976663,310325,ferrous sulfate 325 MG Oral Tablet,0.15,0.0,50,7.50,NaN,NaN
3,2020-10-30 11:02:48+00:00,2020-11-20T11:02:48Z,30a6452c-4297-a1ac-977a-6a23237c7b46,d31fccc3-1767-390d-966a-22a5156f4219,794baa15-fe5e-c061-e188-ad59022aeea5,313782,Acetaminophen 325 MG Oral Tablet,153.58,0.0,1,153.58,NaN,NaN
4,2008-08-27 00:53:03+00:00,2008-08-27T00:53:03Z,34a4dcc4-35fb-6ad5-ab98-be285c586a4f,8fa6c185-e44e-3e34-8bd8-39be8694f4ce,8a84efee-6fd7-f5b3-8816-9a1c60e720be,1535362,sodium fluoride 0.0272 MG/MG Oral Gel,129.94,0.0,1,129.94,66383009.0,Gingivitis (disorder)


### ***Step 2: Join & Merging conditions and medications***

In [ ]:
# Join on PATIENT and ENCOUNTER to create a patient-visit level view
merged_df = pd.merge(conditions, medications, on=['PATIENT', 'ENCOUNTER'], how='inner', suffixes=('_COND', '_MED'))

# Select relevant fields
merged_df = merged_df[[
    'PATIENT', 'ENCOUNTER', 'START_COND', 'DESCRIPTION_COND', 'DESCRIPTION_MED', 'TOTALCOST'
]].rename(columns={
    'START_COND': 'VISIT_DATE',
    'DESCRIPTION_COND': 'DIAGNOSIS',
    'DESCRIPTION_MED': 'MEDICATION',
    'TOTALCOST': 'MEDICATION_COST'
})

# Sort by patient and visit date
merged_df.sort_values(by=['PATIENT', 'VISIT_DATE'], inplace=True)

# View sample
merged_df.head(5)


,PATIENT,ENCOUNTER,VISIT_DATE,DIAGNOSIS,MEDICATION,MEDICATION_COST
170,00732e11-5e4d-37b7-01f8-929a25536862,d66f90b7-2020-4639-0cd5-03e9e8cb61a5,2016-06-29,Stress (finding),insulin isophane human 70 UNT/ML / insulin r...,3835.44
171,00732e11-5e4d-37b7-01f8-929a25536862,d66f90b7-2020-4639-0cd5-03e9e8cb61a5,2016-06-29,Stress (finding),Hydrochlorothiazide 25 MG Oral Tablet,1.80
172,00732e11-5e4d-37b7-01f8-929a25536862,d66f90b7-2020-4639-0cd5-03e9e8cb61a5,2016-06-29,Stress (finding),24 HR tacrolimus 1 MG Extended Release Oral Ta...,519.76
173,00732e11-5e4d-37b7-01f8-929a25536862,d66f90b7-2020-4639-0cd5-03e9e8cb61a5,2016-06-29,Stress (finding),lisinopril 10 MG Oral Tablet,3.64
174,00732e11-5e4d-37b7-01f8-929a25536862,d66f90b7-2020-4639-0cd5-03e9e8cb61a5,2016-06-29,Stress (finding),amLODIPine 2.5 MG Oral Tablet,3.64


### ***Step 3: Create Text-Based Clinical Summaries***

**We’ll convert each row to a short natural language snippet.**

In [ ]:
def generate_summary(row):
    return (f"On {row['VISIT_DATE'].date()}, patient {row['PATIENT']} was diagnosed with "
            f"{row['DIAGNOSIS']} and was prescribed {row['MEDICATION']}. "
            f"The cost of the medication was ${row['MEDICATION_COST']:.2f}.")

merged_df['summary'] = merged_df.apply(generate_summary, axis=1)

# Preview the results
for s in merged_df['summary'].head(3):
    print(s)

On 2016-06-29, patient 00732e11-5e4d-37b7-01f8-929a25536862 was diagnosed with Stress (finding) and was prescribed insulin isophane  human 70 UNT/ML / insulin  regular  human 30 UNT/ML Injectable Suspension [Humulin]. The cost of the medication was $3835.44.
On 2016-06-29, patient 00732e11-5e4d-37b7-01f8-929a25536862 was diagnosed with Stress (finding) and was prescribed Hydrochlorothiazide 25 MG Oral Tablet. The cost of the medication was $1.80.
On 2016-06-29, patient 00732e11-5e4d-37b7-01f8-929a25536862 was diagnosed with Stress (finding) and was prescribed 24 HR tacrolimus 1 MG Extended Release Oral Tablet. The cost of the medication was $519.76.


### ***Step 4: Send Summaries to ChatGPT with Prompt Engineering***

In [ ]:
# Initialize the OpenAI client using your API key
# This should be securely passed (ideally from an environment variable)
client = OpenAI("intentionallyput different text")


# Define a function that uses the OpenAI API to rephrase clinical summaries
def refine_summary_with_llm(summary):
    prompt = f"""
You are a clinical documentation assistant. Rephrase this clinical summary to make it more readable and suitable for patients:
Summary: {summary}
Refined version:"""

    try:
        response = client.chat.completions.create(
          model="gpt-3.5-turbo",  # Use gpt-3.5-turbo if gpt-4 is not available
          messages=[
              {"role": "system", "content": "You are a helpful medical assistant."},
              {"role": "user", "content": prompt}
          ],
          temperature=0.5,
          max_tokens=100
        )
        # Extract and return the model's reply (i.e., the refined summary)
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {e}"


### ***Step 5: Apply Summaries***

In [ ]:
# Apply the LLM refinement function to the first 5 summaries in the DataFrame
# This creates a new column 'refined_summary' with patient-friendly text
merged_df['refined_summary'] = merged_df['summary'].head(5).apply(refine_summary_with_llm)

# Display the original and refined summaries for review
merged_df[['summary', 'refined_summary']].head()

,summary,refined_summary
170,"On 2016-06-29, patient 00732e11-5e4d-37b7-01f8...",Error: Error code: 429 - {'error': {'message':...
171,"On 2016-06-29, patient 00732e11-5e4d-37b7-01f8...",Error: Error code: 429 - {'error': {'message':...
172,"On 2016-06-29, patient 00732e11-5e4d-37b7-01f8...",Error: Error code: 429 - {'error': {'message':...
173,"On 2016-06-29, patient 00732e11-5e4d-37b7-01f8...",Error: Error code: 429 - {'error': {'message':...
174,"On 2016-06-29, patient 00732e11-5e4d-37b7-01f8...",Error: Error code: 429 - {'error': {'message':...
